In [ ]:
import tweepy
import json
import datetime
import time

# API access details - get yours from Twitter
access_token = "1236552411738595328-e1H9aw4mATD4piYGwNuSOQjU6r3M6C"
access_token_secret = "6E8WAnFfYaNOw9vIFQoy6t5EGUsT2NcTX4T3C4DUkuqFu"
consumer_key = "YiiFPmcIiRofzMQbzAW9KVER4"
consumer_secret = "IE8knxJBbFLI2k1kTZMrzLcFpGA6hHB0zUorSwssNfKlx92aD3"
#Authenticate to Twitter api using our account keys
auth_details = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth_details.set_access_token(access_token, access_token_secret)
 
#Create connection to the Twitter api
api = tweepy.API(auth_details)

#Loop and sleep to download tweets
for x in range(0,6):
    tweet_array = []
    tweets = api.home_timeline(count=100)
    for tweet in tweets:
        t = {"text" : tweet.text, "likes": tweet.favorite_count, "retweets": tweet.retweet_count }
        tweet_array.append(t)
        d = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%s')
        f = open('tweets-' + d + ".json", 'w')
        f.write(json.dumps(tweet_array))
        f.close()
    print("Sleeping")
    time.sleep(60 * 30)

Sleeping


In [18]:
import glob
import pathlib
import os
import json

#Make sure we're in the same dir that has the downloaded JSON twitter data
os.chdir('/content')

tweets = []

for file in glob.glob('*.json'):
    with open(file) as json_data:
        data = json.load(json_data)
        for tweet in data:
            tweets.append(tweet)
        
#Show how many tweets we've got
print(len(tweets))

95


In [19]:
from sklearn.neural_network import MLPRegressor
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
import re
from operator import itemgetter
# pre-processing of twitter data
def prepareSentence(s):
    stemmer = LancasterStemmer()
    ignore_words = set(stopwords.words('english'))
    regpattern = re.compile('[\W_]+" "')
    s = re.sub('[^A-Za-z ]+', '', s)
    words = nltk.word_tokenize(s.lower())
    return [stemmer.stem(w.lower()) for w in words if w not in ignore_words]

In [20]:
import nltk
nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
import nltk
nltk.download('punkt')

words = []

for tweet in tweets:
    words.extend(prepareSentence(tweet["text"]))

distinct_words = set(words)

lower_threshold = 10
upper_threshold = 350
counts = []
final_words = []
for word in distinct_words:
    counts.append(words.count(word))
    if words.count(word) > lower_threshold and words.count(word) < upper_threshold: 
        final_words.append(word)

print(len(words))
print(len(distinct_words))
print(len(final_words))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
1039
575
6


In [22]:
# deffination to append all unique words into bag of words
def toBOW(sentance, words):
    bag = []
    for word in words:
        bag.append(1) if word in sentance else bag.append(0)
    return bag

In [23]:
inputs = []
outputs = []

for tweet in tweets:
    sentence = prepareSentence(tweet["text"])
    # create our bag of words array
    bag = toBOW(sentence, final_words)
    inputs.append(bag)
    #Calculate a score, 1 if any engagement, 0 if none
    score = max(tweet["likes"] + tweet["retweets"],1)
    
    
    outputs.append(score)

#Define and train the network
nnet = MLPRegressor(activation='relu', alpha=0.0001, hidden_layer_sizes=(int(len(final_words)*0.5),int(len(final_words)*0.25)),solver='adam', max_iter=400)
nnet.fit(inputs, outputs)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(3, 1), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=400,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [24]:
!pip install feedparser

In [28]:

potential_posts = {}

for tweet in tweets:
        sentence = prepareSentence(tweet["text"])
        bag = toBOW(sentence, final_words)
        potential_posts[tweet["text"]] = nnet.predict([bag])[0]
    

x = sorted(potential_posts.items(), key=itemgetter(1), reverse=True)

print("\nTop post 1:\n")
for i in x[0]:
    print(i)

print("-----------------------------------------------------------------------------------\n")
print("\nTop post 2:\n")
for i in x[1]:
    print(i)

print("-----------------------------------------------------------------------------------\n")
print("\nTop post 3:\n")
for i in x[2]:
    print(i)

print("-----------------------------------------------------------------------------------\n")
print("\nTop post 4:\n")
for i in x[3]:
    print(i)

print("-----------------------------------------------------------------------------------\n")
print("\nTop post 5:\n")
for i in x[4]:
    print(i)



Top post 1:

@MoHFW_INDIA
 issues Revised Discharge Policy for confirmed #COVID19 cases...
-----------------------------------------------------------------------------------


Top post 2:

@drharshvardhan, along with Sh @AshwiniKChoubey, reviews #COVID19 management in eight states...
-----------------------------------------------------------------------------------


Top post 3:

Spitting in public can increase the risk of spread of #COVID19. The onus is on us. Be responsible, be safe...
-----------------------------------------------------------------------------------


Top post 4:

आइये, हम अपने स्वास्थ्य सेवाकर्मियों का ध्यान रखें। महामारी के दौरान उनके साथ किसी भी तरह की हिंसा या संपत्ति को नुकसान पहुंचाना एक दंडनीय अपराध है।...
-----------------------------------------------------------------------------------


Top post 5:

Indians stranded abroad due to the #COVID19 pandemic are getting back home, via air ✈️and sea...
